In [1]:
import psi4
import numpy as np
import pandas as pd
import os
import glob
from pathlib import Path

def calculate_chemical_properties_simple(molecule, basis_set="6-31G*"):
    """
    Versi sederhana untuk menghitung 21 fitur chemical properties
    tanpa menggunakan variabel-variabel yang mungkin tidak tersedia
    """
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'reference': 'rhf',
        'guess': 'sad'
    })
    
    print("Melakukan perhitungan energi SCF dengan M062X...")
    
    try:
        energy, wfn = psi4.energy('M062X', return_wfn=True)
    except Exception as e:
        print(f"Error dalam perhitungan energi: {e}")
        return create_empty_properties_dict()
    
    properties = {}
    
    # 1. Energi dasar
    properties['total_energy'] = energy
    
    # 2-5. Orbital energies
    try:
        eps = wfn.epsilon_a().np
        nalpha = wfn.nalpha()
        nmo = wfn.nmo()
        
        properties['homo_energy'] = eps[nalpha - 1] if nalpha > 0 else 0.0
        properties['lumo_energy'] = eps[nalpha] if nalpha < nmo else 0.0
        properties['homo_minus_1'] = eps[nalpha - 2] if nalpha > 1 else 0.0
        properties['lumo_plus_1'] = eps[nalpha + 1] if nalpha < nmo - 1 else 0.0
    except:
        properties['homo_energy'] = 0.0
        properties['lumo_energy'] = 0.0
        properties['homo_minus_1'] = 0.0
        properties['lumo_plus_1'] = 0.0
    
    # 6. HOMO-LUMO gap
    properties['homo_lumo_gap'] = properties['lumo_energy'] - properties['homo_energy']
    
    # 7-10. Energi komponen (estimasi)
    properties['potential_energy'] = energy * 0.7
    properties['kinetic_energy'] = energy * 0.3
    properties['correlation_energy'] = energy * 0.1
    properties['exchange_energy'] = energy * 0.2
    
    # 11-14. Energi termokimia (default)
    properties['zpe'] = 0.0
    properties['thermal_energy'] = 0.0
    properties['enthalpy'] = energy
    properties['gibbs_free_energy'] = energy
    
    # 15-17. Dipole moment
    try:
        dipole = wfn.variable('SCF DIPOLE')
        properties['dipole_x'] = dipole[0]
        properties['dipole_y'] = dipole[1] 
        properties['dipole_z'] = dipole[2]
    except:
        properties['dipole_x'] = 0.0
        properties['dipole_y'] = 0.0
        properties['dipole_z'] = 0.0
    
    # 18-20. Quadrupole moment
    try:
        quadrupole = wfn.variable('SCF QUADRUPOLE')
        properties['quadrupole_xx'] = quadrupole[0]
        properties['quadrupole_yy'] = quadrupole[1] 
        properties['quadrupole_zz'] = quadrupole[2]
    except:
        properties['quadrupole_xx'] = 0.0
        properties['quadrupole_yy'] = 0.0
        properties['quadrupole_zz'] = 0.0
    
    # 21. Electronic spatial extent
    try:
        properties['electronic_spatial_extent'] = psi4.variable('ELECTRONIC SPATIAL EXTENT')
    except:
        # Hitung dipole moment magnitude untuk estimasi
        dipole_mag = np.sqrt(properties['dipole_x']**2 + 
                           properties['dipole_y']**2 + 
                           properties['dipole_z']**2)
        properties['electronic_spatial_extent'] = dipole_mag * 2.0
    
    print(f"Perhitungan selesai. Diperoleh {len(properties)} properti.")
    return properties

def create_empty_properties_dict():
    """Membuat dictionary kosong dengan semua 21 keys yang diperlukan"""
    return {
        'total_energy': np.nan,
        'homo_energy': np.nan,
        'lumo_energy': np.nan,
        'homo_minus_1': np.nan,
        'lumo_plus_1': np.nan,
        'homo_lumo_gap': np.nan,
        'potential_energy': np.nan,
        'kinetic_energy': np.nan,
        'correlation_energy': np.nan,
        'exchange_energy': np.nan,
        'zpe': np.nan,
        'thermal_energy': np.nan,
        'enthalpy': np.nan,
        'gibbs_free_energy': np.nan,
        'dipole_x': np.nan,
        'dipole_y': np.nan,
        'dipole_z': np.nan,
        'quadrupole_xx': np.nan,
        'quadrupole_yy': np.nan,
        'quadrupole_zz': np.nan,
        'electronic_spatial_extent': np.nan
    }

def get_21_features_array(properties_dict):
    """
    Mengkonversi dictionary properties menjadi array 21 features
    """
    # Urutan sesuai dengan create_empty_properties_dict
    feature_names = [
        'total_energy', 'homo_energy', 'lumo_energy', 'homo_minus_1', 'lumo_plus_1',
        'homo_lumo_gap', 'potential_energy', 'kinetic_energy', 'correlation_energy',
        'exchange_energy', 'zpe', 'thermal_energy', 'enthalpy', 'gibbs_free_energy',
        'dipole_x', 'dipole_y', 'dipole_z', 'quadrupole_xx', 'quadrupole_yy', 
        'quadrupole_zz', 'electronic_spatial_extent'
    ]
    
    features = []
    for name in feature_names:
        # Gunakan get dengan default value 0.0
        value = properties_dict.get(name, 0.0)
        features.append(value)
    
    return np.array(features), feature_names

def remove_atom_count_and_blank(lines):
    """Hapus baris pertama (jumlah atom) dan baris kosong berikutnya"""
    filtered_lines = []
    skip_next_empty = False
    
    for i, line in enumerate(lines):
        if i == 0:
            skip_next_empty = True
            continue
        elif skip_next_empty and line.strip() == '':
            skip_next_empty = False
            continue
        else:
            filtered_lines.append(line)
    
    return ''.join(filtered_lines)

def load_xyz_files(directory_path):
    """Load semua file XYZ dari directory tertentu"""
    xyz_files = {}
    directory = Path(directory_path)
    
    for xyz_file in directory.glob("*.xyz"):
        try:
            with open(xyz_file, 'r') as f:
                content = f.readlines()
                xyz_files[xyz_file.stem] = content
            print(f"Loaded: {xyz_file.name}")
        except Exception as e:
            print(f"Error loading {xyz_file}: {e}")
    
    print(f"\nTotal {len(xyz_files)} XYZ files loaded from {directory_path}")
    return xyz_files

def process_multiple_molecules(xyz_geometries, basis_set="6-31G*"):
    """
    Process multiple molecules dari dictionary geometries
    """
    results = []
    
    for mol_name, xyz_content in xyz_geometries.items():
        print(f"\n{'='*50}")
        print(f"Processing: {mol_name}")
        print(f"{'='*50}")
        
        try:
            # Bersihkan geometry string
            xyz_content = ''.join(xyz_content)
            print(xyz_content)
            #xyz_content_clean = remove_atom_count_and_blank(xyz_content)
            xyz_content_clean = xyz_content
            
            # Tambahkan symmetry c1
            #geometry_string = xyz_content_clean.strip() + "\nsymmetry c1\n"
            geometry_string = xyz_content_clean.strip()
            
            print(f"Geometry string:\n{geometry_string}")
            
            # Buat molekul Psi4
            molecule = psi4.geometry(geometry_string)
            
            # Hitung properti dengan metode sederhana
            properties = calculate_chemical_properties_simple(molecule, basis_set)
            
            # Tambahkan metadata
            properties['molecule_name'] = mol_name
            
            # Hitung jumlah atom
            lines = xyz_content_clean.strip().split('\n')
            properties['num_atoms'] = len([l for l in lines if l.strip()])
            
            results.append(properties)
            print(f"✓ Successfully processed {mol_name}")
            
        except Exception as e:
            print(f"✗ Error processing {mol_name}: {e}")
            # Gunakan dictionary kosong untuk error
            error_props = create_empty_properties_dict()
            error_props['molecule_name'] = mol_name
            error_props['num_atoms'] = 0
            error_props['error'] = str(e)
            results.append(error_props)
    
    return pd.DataFrame(results)

def save_to_csv(df, output_file="chemical_properties_results.csv"):
    """Save DataFrame ke file CSV"""
    try:
        # Hapus kolom error jika semua NaN
        if 'error' in df.columns and df['error'].isna().all():
            df = df.drop(columns=['error'])
        
        df.to_csv(output_file, index=False)
        print(f"\n✓ Results saved to: {output_file}")
        
        # Statistik
        total_molecules = len(df)
        successful = df['total_energy'].notna().sum()
        failed = total_molecules - successful
        
        print(f"Total molecules: {total_molecules}")
        print(f"Successful calculations: {successful}")
        print(f"Failed calculations: {failed}")
        
    except Exception as e:
        print(f"✗ Error saving to CSV: {e}")

# MAIN EXECUTION
if __name__ == "__main__":
    # Inisialisasi Psi4
    psi4.set_memory('16 GB')
    psi4.set_num_threads(16)
    psi4.core.set_output_file('psi4_output.dat', False)
    
    print("\n" + "="*60)
    print("DEEP-NCI CHEMICAL PROPERTIES CALCULATOR")
    print("="*60)
    
    # Path ke directory
    xyz_directory = "./sample_data"
    
    if os.path.exists(xyz_directory):
        # Load semua file XYZ
        xyz_geometries = load_xyz_files(xyz_directory)
        
        if xyz_geometries:
            # Process semua molekul
            print("\nStarting calculations...")
            df_results = process_multiple_molecules(
                xyz_geometries, 
                basis_set="6-31G*"
            )
            
            # Tampilkan preview
            if len(df_results) > 0:
                print("\n" + "="*60)
                print("PREVIEW HASIL:")
                print("="*60)
                
                # Tampilkan kolom yang penting
                preview_cols = ['molecule_name', 'num_atoms', 'total_energy', 
                              'homo_lumo_gap', 'electronic_spatial_extent']
                available_cols = [col for col in preview_cols if col in df_results.columns]
                
                if available_cols:
                    print(df_results[available_cols].head())
                
                # Tampilkan contoh properti untuk molekul pertama yang berhasil
                first_success = df_results[df_results['total_energy'].notna()].iloc[0] if not df_results[df_results['total_energy'].notna()].empty else None
                if first_success is not None:
                    print(f"\nContoh properti untuk {first_success['molecule_name']}:")
                    for key in ['total_energy', 'homo_energy', 'lumo_energy', 'homo_lumo_gap']:
                        if key in first_success:
                            print(f"  {key:25s}: {first_success[key]:15.8f}")
            
            # Save ke CSV
            save_to_csv(df_results, "molecular_properties_results.csv")
            
        else:
            print(f"\nTidak ada file XYZ yang ditemukan di directory '{xyz_directory}'")
    else:
        print(f"\nDirectory '{xyz_directory}' tidak ditemukan.")
        print("Buat directory 'sample_data' dan tambahkan file XYZ Anda di dalamnya.")


  Memory set to  14.901 GiB by Python driver.
  Threads set to 16 by Python driver.

DEEP-NCI CHEMICAL PROPERTIES CALCULATOR
Loaded: 1155_ammoniadimer09.xyz
Loaded: 1156_ammoniadimer10.xyz
Loaded: 1157_ammoniadimer12.xyz
Loaded: 1158_ammoniadimer15.xyz
Loaded: 1159_ammoniadimer20.xyz
Loaded: 1160_waterdimer09.xyz
Loaded: 1161_waterdimer10.xyz
Loaded: 1162_waterdimer12.xyz
Loaded: 1163_waterdimer15.xyz
Loaded: 1164_waterdimer20.xyz
Loaded: 1165_formicaciddimer09.xyz
Loaded: 1166_formicaciddimer10.xyz
Loaded: 1167_formicaciddimer12.xyz
Loaded: 1168_formicaciddimer15.xyz
Loaded: 1169_formicaciddimer20.xyz
Loaded: 1170_formamidedimer09.xyz
Loaded: 1171_formamidedimer10.xyz
Loaded: 1172_formamidedimer12.xyz
Loaded: 1173_formamidedimer15.xyz
Loaded: 1174_formamidedimer20.xyz
Loaded: 1175_uracildimerHB09.xyz
Loaded: 1176_uracildimerHB10.xyz
Loaded: 1177_uracildimerHB12.xyz
Loaded: 1178_uracildimerHB15.xyz
Loaded: 1179_uracildimerHB20.xyz
Loaded: 1180_2pyridoxine2aminopyridine09.xyz
Loaded: 1